# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set Keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

## Primary key for session_id(partition key) and item_in_session(clustering key) since cassandra data model is query based and we look for specific value for sesssionID and itemInSession

## Step 1:  create table(music_artist_title_length) for the artist, song title and song's length in the music app

In [8]:
## Query 1:  create table(music_artist_title_length) for the artist, song title and song's length in the music app

query = "CREATE TABLE IF NOT EXISTS music_artist_title_length "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

## Step 2:  ETL csv files to fitting in music_artist_tile_length for each mapping

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## ETL csv files to fitting in music_artist_tile_length for each mapping
        query = "INSERT INTO music_artist_title_length (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Matching each colum in the INSERT statement for example line[0] for artist_name
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

## Step 3:  Validate the data as artist_name, song_title, and song_length where session_id=338 and item_in_session=4

In [1]:
## Validate the data as artist_name, song_title, and song_length where session_id=338 and item_in_session=4

query = "SELECT artist_name, song_title, song_length FROM music_artist_title_length WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

name 'session' is not defined


NameError: name 'rows' is not defined

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

## Primary Key for (user_id, session_id) because we look for user name results.  Also, item_in_session will be used for clustering and sorting.  Now we combine these two for table design since we look for user / song values.


## Step 1:  Create table(music_artist_song_user) for artist name, song, and user

In [11]:
## Create table(music_artist_song_user) for artist name, song, and user
query = "CREATE TABLE IF NOT EXISTS music_artist_song_user "
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

## Step 2:  ETL process from csv to music_artist_song_user

In [12]:
# read csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## ETL process from csv to music_artist_song_user
        query = "INSERT INTO music_artist_song_user (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

## Step 3:  Validate Data artist_name, song_title_, and name for user_id=10 and session_id=182

In [13]:
## Validate Data artist_name, song_title_, and name for user_id=10 and session_id=182
query = "SELECT artist_name, song_title, first_name, last_name FROM music_artist_song_user WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Primary key for song_title and user_id since we look for specific value for those.  







## Step 1:  Create table(music_user_name) for user_name in my music app history

In [14]:
## Create table(music_user_name) for user_name in my music app history

query = "CREATE TABLE IF NOT EXISTS music_user_name "
query = query + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY((song_title), user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

## Step 2:  ETL csv to music_user_name for each elements.

In [15]:
# read csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## ETL csv to music_user_name for each elements.
        query = "INSERT INTO music_user_name (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

## Step 3:  Validate data for user name where song_title='All Hands Against His Own'

In [16]:
## Validate data for user name where song_title='All Hands Against His Own'
query = "SELECT first_name, last_name FROM music_user_name WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table after completed sessions
drop1 = "drop table music_artist_title_length"
drop2 = "drop table music_artist_song_user"
drop3 = "drop table music_user_name"
try:
    rows = session.execute(drop1)
    rows = session.execute(drop2)
    rows = session.execute(drop3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()